In [ ]:

%pip install --upgrade wheel

In [ ]:
%pip install numpy opencv-python mss playsound pillow

In [ ]:
import numpy as np
import cv2
import mss
import time
from playsound import playsound
from PIL import Image
import os

In [ ]:
class ZoomPollDetector:
    def __init__(self):
        
        self.sct = mss.mss()
        
        self.sound_path = "/Users/mahimamaurya/Desktop/Practice/ring.mp3"
        
        self.poll_template = None
        self.template_path = "/Users/mahimamaurya/Desktop/Practice/Poll.png"
        
        # Cooldown period (in seconds) 
        self.cooldown = 300
        self.last_alert_time = 0

    def capture_template(self):
        """Capture a template of the poll dialog for future detection"""
        print("Please open a poll in Zoom. You have 5 seconds to do so...")
        time.sleep(5)
        
        
        monitor = self.sct.monitors[1]  
        screenshot = self.sct.grab(monitor)
        
        img = Image.frombytes("RGB", screenshot.size, screenshot.rgb)
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        
        cv2.imwrite(self.template_path, img_cv)
        print("Template captured! Press Enter to start monitoring...")
        input()

    def start_monitoring(self):
        """Start monitoring the screen for poll dialogs"""
        print("Monitoring for Zoom polls. Press Ctrl+C to stop.")
        
        
        if os.path.exists(self.template_path):
            self.poll_template = cv2.imread(self.template_path, cv2.IMREAD_COLOR)
        else:
            print("No template found. Capturing new template...")
            self.capture_template()
            self.poll_template = cv2.imread(self.template_path, cv2.IMREAD_COLOR)

        try:
            while True:                
                monitor = self.sct.monitors[1]  
                screenshot = self.sct.grab(monitor)
                
                img = Image.frombytes("RGB", screenshot.size, screenshot.rgb)
                img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                                
                result = cv2.matchTemplate(img_cv, self.poll_template, cv2.TM_CCOEFF_NORMED)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
                
                current_time = time.time()
                if max_val >= 0.8 and (current_time - self.last_alert_time) > self.cooldown:
                    print("Poll detected!")
                    try:
                        playsound(self.sound_path)
                        self.last_alert_time = current_time
                    except Exception as e:
                        print(f"Error playing sound: {e}")
                
                # Add small delay to reduce CPU usage
                time.sleep(0.5)
                
        except KeyboardInterrupt:
            print("\nStopping monitor...")

    def run(self):
        """Main method to run the detector"""
        print("Zoom Poll Detector")
        print("-----------------")
        print("1. Use existing template")
        print("2. Capture new template")
        choice = input("Enter choice (1/2): ")
        
        if choice == "2":
            self.capture_template()
        
        self.start_monitoring()

if __name__ == "__main__":
    detector = ZoomPollDetector()
    detector.run()